In [ ]:
epub_path = '../../Resources/مجنون ليلى.epub'
book_title = 'مجنون ليلى'

## This function is general for any EPUB file

In [ ]:
import zipfile
from bs4 import BeautifulSoup
import os

def extract_and_save_chapters(epub_path, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    with zipfile.ZipFile(epub_path, 'r') as zip_ref:
        zip_ref.extractall("temp_epub_extraction")

    chapter_counter = 1
    for root, dirs, files in sorted(os.walk("temp_epub_extraction")):
        for file in sorted(files):
            if file.endswith('.html') or file.endswith('.xhtml'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    soup = BeautifulSoup(f, 'lxml')
                    texts = soup.get_text(separator='\n', strip=True)
                    chapter_filename = f"chapter_{chapter_counter}.txt"
                    chapter_path = os.path.join(output_directory, chapter_filename)
                    with open(chapter_path, 'w', encoding='utf-8') as chapter_file:
                        chapter_file.write(texts)
                    chapter_counter += 1
    
    # Cleanup extraction directory
    for root, dirs, files in os.walk("temp_epub_extraction", topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir("temp_epub_extraction")

# Specify the directory where you want to save the chapters
output_directory = f'./{book_title}_chapters'

extract_and_save_chapters(epub_path, output_directory)